# Data Cleaning and Preprocessing

In [1]:
import pandas as pd

%matplotlib inline

# Processing the Geocoded Flint Addresses 
Used [census geocoding database](https://geocoding.geo.census.gov/geocoder/geographies/addressbatch?form) to get coordinates from the US Census Bureau . Had to split the batch job in two because of limits on the service. 

In [2]:
names = ['Index','Address', 'Match', 'Exact', 'Address_2', 'Coords', 'Coords_2', 'Coords_3', 'Coords_4', 'Coords_5', 'Coords_6', 'Coords_7']
geocoded_lead_addr_1 = pd.read_csv('./data/GeocodeResults (1).csv', encoding = 'unicode_escape', error_bad_lines = False, names=names, index_col=0)
geocoded_lead_addr_2 = pd.read_csv('./data/GeocodeResults (2).csv', encoding = 'unicode_escape', error_bad_lines= False, names=names, index_col=0)


Dropping addresses that were not successfully geocoded

In [3]:
geocoded_lead_addr_1.dropna(inplace=True, subset=['Coords'])
geocoded_lead_addr_2.dropna(inplace=True, subset=['Coords'])

Concatenating the two dataframes, now we have all of the addresses in one dataframe.
Also had to take care of the Coords column, contained a string representing both latitude and longitude. More convenient to have latitude and longitude in two separate columns

In [4]:
geocoded_lead_addr = pd.concat((geocoded_lead_addr_1, geocoded_lead_addr_2))

latlon = geocoded_lead_addr["Coords"].str.split(",", expand=True)
latlon.columns = ['Lon', 'Lat']

geocoded_lead_addr = geocoded_lead_addr.join(latlon)

geocoded_lead_addr.head()

,Address,Match,Exact,Address_2,Coords,Coords_2,Coords_3,Coords_4,Coords_5,Coords_6,Coords_7,Lon,Lat
Index,,,,,,,,,,,,,
4970,"1608 CROMWELL AVE, FLINT, MI, 48503",Match,Exact,"1608 CROMWELL AVE, FLINT, MI, 48503","-83.66882,43.02084",647468711.0,R,26.0,49.0,3000.0,1022.0,-83.66882,43.02084
2306,"2732 BROWNELL BLVD, FLINT, MI, 48504",Match,Exact,"2732 BROWNELL BLVD, FLINT, MI, 48504","-83.727516,43.041607",69564568.0,R,26.0,49.0,900.0,2007.0,-83.727516,43.041607
2305,"2714 BROWNELL BLVD, FLINT, MI, 48504",Match,Exact,"2714 BROWNELL BLVD, FLINT, MI, 48504","-83.72751,43.0412",69564571.0,R,26.0,49.0,900.0,2015.0,-83.72751,43.0412
2304,"2707 BROWNELL BLVD, FLINT, MI, 48504",Match,Exact,"2707 BROWNELL BLVD, FLINT, MI, 48504","-83.72752,43.04109",69564571.0,L,26.0,49.0,900.0,1013.0,-83.72752,43.04109
2303,"2702 BROWNELL BLVD, FLINT, MI, 48504",Match,Exact,"2702 BROWNELL BLVD, FLINT, MI, 48504","-83.72751,43.041065",69564571.0,R,26.0,49.0,900.0,2015.0,-83.72751,43.041065


Reading in the deduplicated flint water data and joining with the geocoded addresses (on id)

In [5]:
flint_lead_data = pd.read_csv('./data/flint_water_merge_dedup.csv', index_col=0)
flint_lead_data.head()

,Analysis (Copper),Analysis (Lead),City,Copper (ppb),Date Submitted,Lead (ppb),Sample Number,Street #,Street Name,Zip Code
Index,,,,,,,,,,
0,Copper,Lead,FLINT,0.0,2016-04-22 13:27:45,0.0,LG38207,525,12TH ST,48503
1,Copper,Lead,FLINT,0.0,2016-02-26 13:51:00,0.0,LG18552,1907,2ND AVE,48503
2,Copper,Lead,FLINT,260.0,2016-03-06 12:14:59,0.0,LG20757,308,4TH AVE,48503
3,Copper,Lead,FLINT,0.0,2016-02-02 13:28:23,7.0,LG07760,2417,ADAIR ST,48506
4,Copper,Lead,NaN,70.0,2016-02-08 13:05:21,8.0,LG10679,2421,ADAIR ST,48506


In [6]:
lead_data_geo = geocoded_lead_addr.join(flint_lead_data)
with open('lead_data_geo.csv', 'w+') as f:
    lead_data_geo.to_csv(f)

# Acquiring the Flint 2014 Housing Data

This was significantly more difficult than the other

In [7]:
import json
import os

In [8]:
#SEV is State Equalized Value


with open(r'C:\Users\gushi\Projects\buffalo_civic_innovation\hud_arcgis_download_script\hud_file_list') as g:
    hudfiles = g.read()

hudfiles = hudfiles.split()

for hudfile in hudfiles:
    #print(r'C:\Users\gushi\Projects\buffalo_civic_innovation\hud_arcgis_download_script\' + hudfile)
    with open(os.path.join(r'C:\Users\gushi\Projects\buffalo_civic_innovation\hud_arcgis_download_script', hudfile)) as f:
        hud_dict = json.load(f)
    for hud_prop in hud_dict["features"]:
        if "2714 BROWNELL BLVD" in hud_prop['attributes']['Prop_Addr']:
            print(hud_prop['attributes']['Prop_Addr'], hud_prop['attributes']['lowmod_pct'])

#for hud_prop in hud_dict["features"]:
 #   print(hud_prop['attributes']['Prop_Addr'])

2714 BROWNELL BLVD 0.6731


Use Zillow API to get the property age and value!
Looked at fire insurance maps, county clerk etc. No dice. 

# Using Zillow API to access home age and home value

In [10]:
import requests


key = os.environ['ZWSID']
base_url = 'http://www.zillow.com/webservice/GetDeepSearchResults.htm'
params = {'zws-id':key, 'address':'2714 BROWNELL BLVD', 'citystatezip':'FLINT MI 48504'}

r = requests.get(base_url, params=params)


'<?xml version="1.0" encoding="utf-8"?><SearchResults:searchresults xsi:schemaLocation="http://www.zillow.com/static/xsd/SearchResults.xsd https://www.zillowstatic.com/vstatic/7de9b24/static/xsd/SearchResults.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:SearchResults="http://www.zillow.com/static/xsd/SearchResults.xsd"><request><address>2714 BROWNELL BLVD</address><citystatezip>FLINT MI 48504</citystatezip></request><message><text>Request successfully processed</text><code>0</code></message><response><results><result><zpid>73942912</zpid><links><homedetails>https://www.zillow.com/homedetails/2714-Brownell-Blvd-Flint-MI-48504/73942912_zpid/</homedetails><graphsanddata>http://www.zillow.com/homedetails/2714-Brownell-Blvd-Flint-MI-48504/73942912_zpid/#charts-and-data</graphsanddata><mapthishome>http://www.zillow.com/homes/73942912_zpid/</mapthishome><comparables>http://www.zillow.com/homes/comps/73942912_zpid/</comparables></links><address><street>2714 Brownell Blvd</s

In [30]:
from xml.etree import ElementTree
import xmltodict

resp_dict = xmltodict.parse(r.text)
resp_dict

OrderedDict([('SearchResults:searchresults',
              OrderedDict([('@xsi:schemaLocation',
                            'http://www.zillow.com/static/xsd/SearchResults.xsd https://www.zillowstatic.com/vstatic/7de9b24/static/xsd/SearchResults.xsd'),
                           ('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('@xmlns:SearchResults',
                            'http://www.zillow.com/static/xsd/SearchResults.xsd'),
                           ('request',
                            OrderedDict([('address', '2714 BROWNELL BLVD'),
                                         ('citystatezip', 'FLINT MI 48504')])),
                           ('message',
                            OrderedDict([('text',
                                          'Request successfully processed'),
                                         ('code', '0')])),
                           ('response',
                            OrderedD

In [34]:
resp_dict['SearchResults:searchresults']['response']['results']['result']['address']
resp_dict['SearchResults:searchresults']['response']['results']['result']['address']['street']
resp_dict['SearchResults:searchresults']['response']['results']['result']['address']['zipcode']
resp_dict['SearchResults:searchresults']['response']['results']['result']['address']['city']
resp_dict['SearchResults:searchresults']['response']['results']['result']['address']['state']
resp_dict['SearchResults:searchresults']['response']['results']['result']['address']['latitude']
resp_dict['SearchResults:searchresults']['response']['results']['result']['address']['longitude']
resp_dict['SearchResults:searchresults']['response']['results']['result']['useCode']
resp_dict['SearchResults:searchresults']['response']['results']['result']['taxAssessmentYear']
resp_dict['SearchResults:searchresults']['response']['results']['result']['taxAssessment']
resp_dict['SearchResults:searchresults']['response']['results']['result']['yearBuilt']

'SingleFamily'